<a href="https://colab.research.google.com/github/faisal-ba-systems/ML-course-documents/blob/main/EDA_SDC_Business_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
import seaborn as sns

In [ ]:
!pip install -q gdown

# 22 July 2025
# https://docs.google.com/spreadsheets/d/1MY2C35RoUXac6zl2k79gmrEeB-vsy4W4qZEoGpSLnyQ/edit?usp=sharing
# !gdown 1MY2C35RoUXac6zl2k79gmrEeB-vsy4W4qZEoGpSLnyQ

# 14 August 2025
# https://docs.google.com/spreadsheets/d/12p_j5_sAhfvnNOn3Pa8aPPgLkknVHV3Y3qMWk_ljkns/edit?usp=sharing
! gdown 12p_j5_sAhfvnNOn3Pa8aPPgLkknVHV3Y3qMWk_ljkns

Downloading...
From (original): https://drive.google.com/uc?id=12p_j5_sAhfvnNOn3Pa8aPPgLkknVHV3Y3qMWk_ljkns
From (redirected): https://docs.google.com/spreadsheets/d/12p_j5_sAhfvnNOn3Pa8aPPgLkknVHV3Y3qMWk_ljkns/export?format=xlsx
To: /content/SDC EDA Course Data 14 August 2025.xlsx
675kB [00:00, 8.29MB/s]


In [ ]:
excel_path ='/content/SDC EDA Course Data 14 August 2025.xlsx'
sheet_name = 'batch status'
sheet_name_speaker = 'speaker list'
cluster_course='course cluster'
df = pd.read_excel(excel_path, sheet_name=sheet_name)
speaker_df = pd.read_excel(excel_path, sheet_name=sheet_name_speaker)
cluster_df = pd.read_excel(excel_path, sheet_name=cluster_course)

In [ ]:
def report_data_types_uniques_check(df):
    col = []
    d_type = []
    uniques = []
    n_uniques = []

    for i in df.columns:
        col.append(i)
        d_type.append(df[i].dtypes)
        uniques.append(df[i].unique()[:5])
        n_uniques.append(df[i].nunique())

    return pd.DataFrame({'Column': col, 'd_type': d_type, 'unique_sample': uniques, 'n_uniques': n_uniques})

# report_data_types_uniques_check(df)
# report_data_types_uniques_check(speaker_df)
# report_data_types_uniques_check(cluster_df)

### Analysis Dataset

In [ ]:
batch_color_discrete_map={
                'complete': 'green',
                'ongoing': 'gold',
                'upcoming': 'lightgreen',
                 }

certificate_color_discrete_map={
                'Certified': 'green',
                'Not certified': 'red',
                'Not Certified': 'red',
              }

gender_color_discrete_map={
                'male': 'green',
                'female': 'purple',
              }

participant_type_color_map = {
    'Internal': 'purple',
    'External': 'lightgreen',
}

approval_status_color_map = {
    'Approved': 'green',
    'Not Approved': 'red',
}

# Convert all to lowercase
df["Gender"] = df["Gender"].str.lower()


# time to decimal
def time_to_decimal(time_str):
    if isinstance(time_str, str) and ':' in time_str:
        parts = time_str.strip().split(':')  # remove whitespace, then split
        try:
            parts = list(map(int, parts))  # convert all parts to integers
            if len(parts) == 2:
                hours, minutes = parts
                seconds = 0
            elif len(parts) == 3:
                hours, minutes, seconds = parts
            else:
                return None  # Unexpected number of parts
            return hours + minutes / 60 + seconds / 3600
        except ValueError:
            return None  # Non-numeric values
    return None


### Course Data Overview

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Create subplots with 2 rows and 2 columns
fig = make_subplots(
    rows=6, cols=2,
    subplot_titles=( "Course Distribution", "Course Distribution",
                     "Participant Type", "Unique Participant Distribution",
                     "Participant Gender", "Participant Gender Distribution",
                     "Speaker Gender", "Speaker Gender Distribution",
                     "Participants Approval Status Based on Type", 'Participant Approval Distribution',
                     "Certificate Status", "Certificate Status Distribution",
 ),
    specs=[[{"type": "table"}, {"type": "pie"}],
           [{"type": "table"}, {"type": "pie"}],
           [{"type": "table"}, {"type": "pie"}],
           [{"type": "table"}, {"type": "pie"}],
           [{"type": "table"}, {"type": "pie"}],
           [{"type": "table"}, {"type": "pie"}]
        ],
    vertical_spacing=0.08,
    horizontal_spacing=0.2,

)
############### Course Duration - Table - 1 - start  ###############

# Convert 'CourseDuration' to decimal hours in df
df['CourseDurationDecimal'] = df['CourseDuration'].apply(time_to_decimal)

# First, get the duration for each individual course (use unique since each course should have same duration)
course_durations = df.groupby('coursename')['CourseDurationDecimal'].first().reset_index()

# Get unique course-batch status mapping
course_batch_mapping = df[['coursename', 'batch status']].drop_duplicates()

# Merge and calculate sum of individual course durations for each batch status
batch_duration_totals = course_durations.merge(course_batch_mapping, on='coursename')
batch_duration_totals = batch_duration_totals.groupby('batch status')['CourseDurationDecimal'].sum().reset_index()

# Group by 'batch status' and count distinct course names
course_status_counts = df.groupby('batch status')['coursename'].nunique().reset_index()

# Merge the counts with duration totals
course_status_counts = course_status_counts.merge(batch_duration_totals, on='batch status')

# Rename columns for clarity
course_status_counts.rename(columns={
    'coursename': 'Count',
    'CourseDurationDecimal': 'TotalDuration'
}, inplace=True)

# Calculate totals
total_count = course_status_counts['Count'].sum()
total_duration = course_status_counts['TotalDuration'].sum()  # Fixed: Added .sum().sum()

# Add the "Total" row dynamically for the table
total_row = pd.DataFrame({
    'batch status': ['Total'],
    'Count': [total_count],
    'TotalDuration': [total_duration]
})
course_status_counts_with_total = pd.concat([course_status_counts, total_row], ignore_index=True)

# Format duration for display (round to 1 decimal place)
course_status_counts_with_total['TotalDurationFormatted'] = course_status_counts_with_total['TotalDuration'].round(1)

# Prepare data for the table (Batch Status, Course Duration, and Course Count) including "Total" row
table_data = [
    course_status_counts_with_total['batch status'].tolist(),
    course_status_counts_with_total['Count'].tolist(),
    course_status_counts_with_total['TotalDurationFormatted'].tolist()  # Duration in decimal hours

]

# Add the table to the left subplot (Row 1, Column 1)
fig.add_trace(go.Table(
    header=dict(
        values=["Batch Status", "Number of Courses", "Course Duration (Hours)"],
        fill_color='burlywood',
        align='center',
        line_color='black',
        font=dict(size=16),
        line_width=2,
        height=40
    ),
    cells=dict(
        values=table_data,
        fill_color='white',
        align='center',
        line_color='black',
        font=dict(size=14),
        line_width=2,
        height=35
    )
), row=1, col=1)

# Remove the "Total" row for the pie chart (exclude it from the data used in the pie chart)
course_status_counts_without_total = course_status_counts.copy()

# Add the pie chart to the right subplot (Row 1, Column 2)
fig.add_trace(go.Pie(
    labels=course_status_counts_without_total['batch status'],
    values=course_status_counts_without_total['Count'],
    hole=0.3,
    # marker=dict(colors=['green', 'yellow', 'blue']),
    marker=dict(colors=[batch_color_discrete_map.get(s, 'gray') for s in course_status_counts_without_total['batch status']]),

), row=1, col=2)

############### Course Duration - Table - 1 - End  ###############


# ############### Paricipant Type - Table - 2 - Start  ###############

# Step 1: Convert CourseDuration to decimal hours
df['CourseDurationDecimal'] = df['CourseDuration'].apply(time_to_decimal)

# Step 2: Aggregate by Type (Internal/External)
participant_type_df = df.groupby('Type').agg({
    'Email': pd.Series.nunique,                 # Unique participants
    'CourseDurationDecimal': 'sum'             # Total course hours
}).reset_index()

# Step 3: Rename columns
participant_type_df.columns = ['Participant Type', 'Number of Unique Participants', 'Course Duration (hours)']

# Step 4: Add Total row
total_row = pd.DataFrame({
    'Participant Type': ['Total'],
    'Number of Unique Participants': [participant_type_df['Number of Unique Participants'].sum()],
    'Course Duration (hours)': [participant_type_df['Course Duration (hours)'].sum()]
})
participant_type_df_total = pd.concat([participant_type_df, total_row], ignore_index=True)

# Step 5: Round duration for display
participant_type_df_total['Course Duration (hours)'] = participant_type_df_total['Course Duration (hours)'].round(1)

# Step 6: Prepare Table Data
table_data = [
    participant_type_df_total['Participant Type'].tolist(),
    participant_type_df_total['Number of Unique Participants'].tolist(),
    participant_type_df_total['Course Duration (hours)'].tolist()
]

# Step 7: Add Table to Subplot
fig.add_trace(
    go.Table(
        header=dict(
            values=["Participant Type", "Number of Unique Participants", "Total Course Duration (hours)"],
            fill_color='burlywood',
            align='center',
            line_color='black',
            font=dict(size=16),
            line_width=2,
            height=40
        ),
        cells=dict(
            values=table_data,
            fill_color='white',
            align='center',
            line_color='black',
            font=dict(size=14),
            line_width=2,
            height=35
        )
    ),
    row=2, col=1  # You can change the row/col based on your layout
)

# Step 8: Optional - Pie Chart for Unique Participants only (excluding total)
pie_df = participant_type_df.copy()
fig.add_trace(
    go.Pie(
        labels=pie_df['Participant Type'],
        values=pie_df['Number of Unique Participants'],
        hole=0.3,
        marker=dict(colors=[participant_type_color_map.get(s, 'gray') for s in pie_df['Participant Type']])
    ),
    row=2, col=2
)
############### Paricipant Type - Table - 2 - End  ###############
############### Participant Gender- Table - 3 - Start  ###############


# Group by gender and count unique emails
gender_status_counts = df.groupby('Gender')['Email'].nunique().reset_index()
gender_status_counts.columns = ['Gender', 'Count']

# Add "Total" row for the table
total_count = gender_status_counts['Count'].sum()
total_row = pd.DataFrame({'Gender': ['Total'], 'Count': [total_count]})
gender_status_counts_with_total = pd.concat([gender_status_counts, total_row], ignore_index=True)

table_data_2 = [
    gender_status_counts_with_total['Gender'].tolist(),
    gender_status_counts_with_total['Count'].tolist()
]

fig.add_trace(go.Table(
    header=dict(
        values=["Gender", "Unique Participants"],
        fill_color='burlywood',
        align='center',
        line_color='black',
        font=dict(size=14),
        line_width=2,
        height=35
    ),
    cells=dict(
        values=table_data_2,
        fill_color='white',
        align='center',
        line_color='black',
        font=dict(size=12),
        line_width=2,
        height=30
    )
), row=3, col=1)

# Exclude the "Total" row from the pie chart
gender_status_counts_without_total = gender_status_counts[gender_status_counts['Gender'] != 'Total']

fig.add_trace(go.Pie(
    labels=gender_status_counts_without_total['Gender'],
    values=gender_status_counts_without_total['Count'],
    hole=0.3,
    # marker=dict(colors=['blue', 'pink']),
    marker=dict(colors=[gender_color_discrete_map.get(s, 'gray') for s in gender_status_counts_without_total['Gender']]),

), row=3, col=2)

############### Participant Gender- Table - 3 - Start  ###############

############### Speker Gender- Table - 4 - Start  ###############

# Group by gender and count unique emails
gender_status_counts = speaker_df.groupby('gender')['email'].nunique().reset_index()
gender_status_counts.columns = ['gender', 'Count']

# Add "Total" row for the table
total_count = gender_status_counts['Count'].sum()
total_row = pd.DataFrame({'gender': ['Total'], 'Count': [total_count]})
gender_status_counts_with_total = pd.concat([gender_status_counts, total_row], ignore_index=True)

table_data_2 = [
    gender_status_counts_with_total['gender'].tolist(),
    gender_status_counts_with_total['Count'].tolist()
]

fig.add_trace(go.Table(
    header=dict(
        values=["Gender", "Unique Speaker"],
        fill_color='burlywood',
        align='center',
        line_color='black',
        font=dict(size=14),
        line_width=2,
        height=35
    ),
    cells=dict(
        values=table_data_2,
        fill_color='white',
        align='center',
        line_color='black',
        font=dict(size=12),
        line_width=2,
        height=30
    )
), row=4, col=1)

# Exclude the "Total" row from the pie chart
gender_status_counts_without_total = gender_status_counts[gender_status_counts['gender'] != 'Total']

fig.add_trace(go.Pie(
    labels=gender_status_counts_without_total['gender'],
    values=gender_status_counts_without_total['Count'],
    hole=0.3,
    # marker=dict(colors=['blue', 'pink']),
    marker=dict(colors=[gender_color_discrete_map.get(s, 'gray') for s in gender_status_counts_without_total['gender']]),

), row=4, col=2)

############### Speker Gender- Table - 4 - End  ###############

############### Approval- Table - 5 - Start  ###############
# Step 1: Create a pivot table with Status and Type counts
status_type_counts = df.pivot_table(
    index='Status',
    columns='Type',
    values='Email',
    aggfunc='count',
    fill_value=0
).reset_index()

# Step 2: Add Total columns and rows
status_type_counts['Total'] = status_type_counts.get('External', 0) + status_type_counts.get('Internal', 0)

# Add total row
total_row = pd.DataFrame([{
    'Status': 'Total',
    'External': status_type_counts.get('External', 0).sum(),
    'Internal': status_type_counts.get('Internal', 0).sum(),
    'Total': status_type_counts['Total'].sum()
}])

# Combine
status_type_summary = pd.concat([status_type_counts, total_row], ignore_index=True)

# Step 3: Prepare table data
table_data = [
    status_type_summary['Status'].tolist(),
    status_type_summary.get('External', pd.Series(dtype=int)).tolist(),
    status_type_summary.get('Internal', pd.Series(dtype=int)).tolist(),
    status_type_summary['Total'].tolist()
]

# Step 4: Draw Table in Plotly
fig.add_trace(
    go.Table(
        header=dict(
            values=["Approval Status", "External", "Internal", "Total"],
            fill_color='burlywood',
            align='center',
            line_color='black',
            font=dict(size=16),
            line_width=2,
            height=40
        ),
        cells=dict(
            values=table_data,
            fill_color='white',
            align='center',
            line_color='black',
            font=dict(size=14),
            line_width=2,
            height=35
        )
    ),
    row=5, col=1
)

# (Optional) Pie Chart still uses just total status counts
approval_status_counts = df['Status'].value_counts().reset_index()
approval_status_counts.columns = ['Status', 'Count']
pie_df = approval_status_counts.copy()

fig.add_trace(
    go.Pie(
        labels=pie_df['Status'],
        values=pie_df['Count'],
        hole=0.3,
        marker=dict(colors=[approval_status_color_map.get(s, 'gray') for s in pie_df['Status']]),
        textinfo='label+percent',
        textposition='inside'
    ),
    row=5, col=2
)



############### Approval- Table - 5 - End  ###############

# certificate_status_counts = certificate_status_counts[certificate_status_counts['Status'] != 'Approved']
############### Certificate- Table - 6 - Start  ###############

# Table : Certificate Status (Row 1, Col 1)
# Filter only Approved status
approved_df = df[df['Status'] == 'Approved']

certificate_status_counts = approved_df['certificate gain'].value_counts().reset_index()
certificate_status_counts.columns = ['certificate gain', 'Count']

total_count = certificate_status_counts['Count'].sum()
total_row = pd.DataFrame({'certificate gain': ['Total'], 'Count': [total_count]})
certificate_status_counts_with_total = pd.concat([certificate_status_counts, total_row], ignore_index=True)

table_data_1 = [
    certificate_status_counts_with_total['certificate gain'].tolist(),
    certificate_status_counts_with_total['Count'].tolist()
]

fig.add_trace(go.Table(
    header=dict(
        values=["Certificate Status", "Count"],
        fill_color='burlywood',
        align='center',
        line_color='black',
        font=dict(size=14),
        line_width=2,
        height=35
    ),
    cells=dict(
        values=table_data_1,
        fill_color='white',
        align='center',
        line_color='black',
        font=dict(size=12),
        line_width=2,
        height=30
    )
), row=6, col=1)

# Exclude the "Total" row from the pie chart
certificate_status_counts_without_total = certificate_status_counts[certificate_status_counts['certificate gain'] != 'Total']

fig.add_trace(go.Pie(
    labels=certificate_status_counts_without_total['certificate gain'],
    values=certificate_status_counts_without_total['Count'],
    hole=0.3,
    marker=dict(colors=['green', 'red']),
    # marker=dict(colors=[certificate_color_discrete_map.get(s, 'gray') for s in certificate_status_counts['certificate gain']])
), row=6, col=2)

############### Certificate- Table - 6 - End  ###############

# Update layout
fig.update_layout(
    title_text="SDC Data Overview Dashboard",
    title_x=0.5,
     title=dict(
        text=" <b>SDC Data Overview Dashboard</b> <br> Business Automation Ltd. <br>",
        x=0.5,  # Center the title
        xanchor='center',
        y=0.99,  # Position title higher
        yanchor='top',
        pad=dict(b=200),  # Add padding below the title
        font=dict(size=25)
    ),

    showlegend=False,
    height=2600,  # Increased height for better readability
    width=1400,  # Increased width for better readability
    plot_bgcolor='white'
)

# Show the plot
fig.show()

### Number of Courses in Each Category in SDC

In [ ]:
import pandas as pd
import plotly.express as px

# Group by cluster and aggregate course names with <br> separator for vertical display
cluster_course_names = cluster_df.groupby('cluster')['coursename'].agg(lambda x: '<br>'.join(sorted(set(x)))).reset_index()
cluster_course_names.columns = ['Cluster', 'Course Names']

# Count number of courses per cluster
cluster_course_counts = cluster_df.groupby('cluster')['coursename'].count().reset_index()
cluster_course_counts.columns = ['Cluster', 'Course Count']

# Merge course names into course count table
cluster_course_counts = pd.merge(cluster_course_counts, cluster_course_names, on='Cluster')

# Sort by Course Count ascending (for horizontal bar chart)
cluster_course_counts = cluster_course_counts.sort_values(by='Course Count', ascending=True)

# Add index as ranking order (for coloring)
cluster_course_counts['Rank'] = cluster_course_counts['Course Count'].rank(method='first', ascending=False)

# Plot bar chart with vertical course list on hover
fig = px.bar(cluster_course_counts,
             x='Course Count',
             y='Cluster',
             title='Chart 2: Analysis on Training Program Categorization under SDC',
             text='Course Count',
             color='Course Count',
             color_continuous_scale='RdYlGn',
             hover_data={
                 'Cluster': True,
                 'Course Count': True,
                 'Course Names': True,
                 'Rank': False
             })

fig.update_traces(
    textposition='inside',
    hovertemplate='<b>Cluster:</b> %{y}<br>' +
                  '<b>Course Count:</b> %{x}<br>' +
                  '<b>Course Names:</b><br>%{customdata[0]}<extra></extra>'
)

fig.update_layout(
    xaxis_title='Cluster',
    yaxis_title='Number of Courses',
    showlegend=False,
    hoverlabel=dict(bgcolor="white", font_size=13, font_family="Arial")
)

fig.show()


### Hidden=> Investment Overview: Participants vs Speakers

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd

# Assuming 'df' and 'speaker_df' are already available.

# Convert 'CourseDuration' to decimal hours in df
df['CourseDurationDecimal'] = df['CourseDuration'].apply(time_to_decimal)

# Filter completed courses and where 'Type' is 'Internal' and 'certificate gain' is 'Certified'
completed_courses_df_speaker = df[(df['batch status'] != 'upcoming')]
completed_courses_df_participant = df[(df['batch status'] != 'upcoming') & (df['Type'] == 'Internal') & (df['certificate gain'] == 'Certified')]

# Merge df with speaker_df on 'speaker_id' and 'id' to get speaker names
merged_df_speaker = pd.merge(completed_courses_df_speaker, speaker_df, left_on='speaker_id', right_on='id', how='left')
merged_df_participant = pd.merge(completed_courses_df_participant, speaker_df, left_on='speaker_id', right_on='id', how='left')


# Prepare the data for participant fees (based on courses and participants)
participant_courses_duration = merged_df_participant.groupby(['email', 'name', 'coursename'])['CourseDurationDecimal'].sum().reset_index()
participant_courses_duration['CourseFeeparticipant'] = participant_courses_duration['CourseDurationDecimal'] * 1000
participant_fee = participant_courses_duration.groupby(['email', 'name'])['CourseFeeparticipant'].sum().reset_index()
participant_fee.columns = ['Speaker Email', 'Speaker Name', 'Total Investment (Taka)']
participant_fee_sorted = participant_fee.sort_values(by='Total Investment (Taka)', ascending=False).head(20)

# Prepare the data for speaker fees (based on unique courses and durations)
unique_courses_duration = merged_df_speaker.groupby(['email', 'name', 'coursename'])['CourseDurationDecimal'].first().reset_index()
unique_courses_duration['CourseFee'] = unique_courses_duration['CourseDurationDecimal'] * 1000
speaker_fee = unique_courses_duration.groupby(['email', 'name'])['CourseFee'].sum().reset_index()
speaker_fee.columns = ['Speaker Email', 'Speaker Name', 'Total Investment (Taka)']
speaker_fee_sorted = speaker_fee.sort_values(by='Total Investment (Taka)', ascending=False).head(20)

# Create a 2x2 subplot for the tables and pie chart
fig = sp.make_subplots(
    rows=2, cols=2,
    subplot_titles=('BA Participants Investment (Top 20)', 'Speakers Investment (Top 20)',
                    'Total Investment Overview', 'Total Investment Comparison'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}], [{'type': 'table'}, {'type': 'pie'}]],
    column_widths=[0.45, 0.55],  # Adjust the column widths
    row_heights=[0.45, 0.55],  # Adjust the row heights
)

# Add the Participant Investment chart (bar chart) with continuous color scale
fig.add_trace(
    go.Bar(
        x=participant_fee_sorted['Speaker Name'],
        y=participant_fee_sorted['Total Investment (Taka)'],
        marker=dict(
            color=participant_fee_sorted['Total Investment (Taka)'],  # Apply continuous color scale based on the value
            colorscale='RdYlGn',  # Set color scale (Red to Green)
            # colorbar=dict(title='Investment (Taka)')
        ),
        name='BA Participants Investment',
        hovertemplate="<b>Speaker Name:</b> %{x}<br><b>Email:</b> %{customdata[0]}<br><b>Investment:</b> %{y}<extra></extra>",
        customdata=participant_fee_sorted[['Speaker Email']]  # Add the email as custom data for hover
    ),
    row=1, col=1
)

# Add the Speaker Investment chart (bar chart) with continuous color scale
fig.add_trace(
    go.Bar(
        x=speaker_fee_sorted['Speaker Name'],
        y=speaker_fee_sorted['Total Investment (Taka)'],
        marker=dict(
            color=speaker_fee_sorted['Total Investment (Taka)'],  # Apply continuous color scale based on the value
            colorscale='RdYlGn',  # Set color scale (Red to Green)
            # colorbar=dict(title='Investment (Taka)')
        ),
        name='Speakers Investment',
        hovertemplate="<b>Speaker Name:</b> %{x}<br><b>Email:</b> %{customdata[0]}<br><b>Investment:</b> %{y}<extra></extra>",
        customdata=speaker_fee_sorted[['Speaker Email']]  # Add the email as custom data for hover
    ),
    row=1, col=2
)

# Create the total investment comparison table and pie chart
# Calculate total fees for participants and speakers
total_participant_fee = participant_fee['Total Investment (Taka)'].sum()
total_speaker_fee = speaker_fee['Total Investment (Taka)'].sum()
grand_total = total_participant_fee + total_speaker_fee

# Create separate DataFrames for table and pie chart
# Table DataFrame (includes total)
table_df = pd.DataFrame({
    'Category': ['Participants', 'Speakers', 'Total'],
    'Total Investment (Taka)': [total_participant_fee, total_speaker_fee, grand_total]
})

# Pie chart DataFrame (excludes total)
pie_df = pd.DataFrame({
    'Category': ['Participants', 'Speakers'],
    'Total Investment (Taka)': [total_participant_fee, total_speaker_fee]
})

# Create the table for the fee comparison
table_fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Category", "Total Investment (Taka)"],
        fill_color='burlywood',
        align='center',
        line_color='black',
        font=dict(size=16),
        line_width=2,
        height=40
    ),
    cells=dict(
        values=[table_df['Category'], table_df['Total Investment (Taka)']],
        fill_color='white',
        line_color='black',
        font=dict(size=14),
        line_width=2,
        height=35,
        align='center'
    )
)])

# Create the pie chart for the fee comparison (without total)
pie_fig = px.pie(pie_df,
                 names='Category',
                 values='Total Investment (Taka)',
                 title="Total Investment Comparison: Participants vs Speakers",
                 color='Category',
                 color_discrete_map={'Participants': 'lightgreen', 'Speakers': 'purple'})

# Add the table to the bottom-left (row 2, col 1)
fig.add_trace(table_fig.data[0], row=2, col=1)

# Add the pie chart to the bottom-right (row 2, col 2)
fig.add_trace(pie_fig['data'][0], row=2, col=2)

# Update layout for the final plot
fig.update_layout(
    title="Chart 3: Investment Overview: <b>Participants vs Speakers</b> <br> (1 hour = 1000 Taka)",
    title_x=0.5,
    # title_y=0.98,
    height=1000,  # Increased height to fit all the content
    width=1500,  # Adjust the width for better readability
    showlegend=False,
    plot_bgcolor='white',
    margin=dict(l=50, r=50, b=50),  # Add some margin around the plot
)

# Show the final plot
fig.show()

## Investment Overview: <b>Participants vs Speakers</b>

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd

# Convert 'CourseDuration' to decimal hours in df
df['CourseDurationDecimal'] = df['CourseDuration'].apply(time_to_decimal)

# Filter
completed_courses_df_speaker = df[(df['batch status'] != 'upcoming')]
completed_courses_df_participant = df[
    (df['batch status'] != 'upcoming') & (df['Type'] == 'Internal') & (df['certificate gain'] == 'Certified')
]

# Merge to get speaker names
merged_df_speaker = pd.merge(completed_courses_df_speaker, speaker_df, left_on='speaker_id', right_on='id', how='left')
merged_df_participant = pd.merge(completed_courses_df_participant, speaker_df, left_on='speaker_id', right_on='id', how='left')

# Participant investment (top 20)
participant_courses_duration = (
    merged_df_participant.groupby(['email', 'name', 'coursename'])['CourseDurationDecimal'].sum().reset_index()
)
participant_courses_duration['CourseFeeparticipant'] = participant_courses_duration['CourseDurationDecimal'] * 1000
participant_fee = (
    participant_courses_duration.groupby(['email', 'name'])['CourseFeeparticipant'].sum().reset_index()
)
participant_fee.columns = ['Speaker Email', 'Speaker Name', 'Total Investment (Taka)']
participant_fee_sorted = participant_fee.sort_values(by='Total Investment (Taka)', ascending=False).head(20)

# Speaker fees (kept for table & pie ONLY)
unique_courses_duration = (
    merged_df_speaker.groupby(['email', 'name', 'coursename'])['CourseDurationDecimal'].first().reset_index()
)
unique_courses_duration['CourseFee'] = unique_courses_duration['CourseDurationDecimal'] * 1000
speaker_fee = unique_courses_duration.groupby(['email', 'name'])['CourseFee'].sum().reset_index()
speaker_fee.columns = ['Speaker Email', 'Speaker Name', 'Total Investment (Taka)']
speaker_fee_sorted = speaker_fee.sort_values(by='Total Investment (Taka)', ascending=False).head(20)

# Totals (for table & pie)
total_participant_fee = participant_fee['Total Investment (Taka)'].sum()
total_speaker_fee = speaker_fee['Total Investment (Taka)'].sum()
grand_total = total_participant_fee + total_speaker_fee

table_df = pd.DataFrame({
    'Category': ['Participants', 'Speakers', 'Total'],
    'Total Investment (Taka)': [total_participant_fee, total_speaker_fee, grand_total]
})
pie_df = pd.DataFrame({
    'Category': ['Participants', 'Speakers'],
    'Total Investment (Taka)': [total_participant_fee, total_speaker_fee]
})

# ----- LAYOUT: top cell spans two columns; bottom-left table, bottom-right pie -----
fig = sp.make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'bar', 'colspan': 2}, None],
           [{'type': 'table'}, {'type': 'pie'}]],
    subplot_titles=(
        'BA Participants Investment (for Top 20 speaker)',  # row1 col1-2
        '',                                      # placeholder for the spanned cell
        'Investment Summary',                    # row2 col1
        'Total Investment Comparison'            # row2 col2
    )
)

# Top (FULL WIDTH): Participants bar (continuous colors)
fig.add_trace(
    go.Bar(
        x=participant_fee_sorted['Speaker Name'],
        y=participant_fee_sorted['Total Investment (Taka)'],
        marker=dict(
            color=participant_fee_sorted['Total Investment (Taka)'],
            colorscale='RdYlGn'
        ),
        name='BA Participants Investment',
        text=participant_fee_sorted['Total Investment (Taka)'],
        textposition='outside',
        hovertemplate="<b>Speaker:</b> %{x}<br><b>Investment:</b> %{y}<br><b>Email:</b> %{customdata[0]}<extra></extra>",
        customdata=participant_fee_sorted[['Speaker Email']]
    ),
    row=1, col=1
)

# NOTE: NO speaker bar chart added.

# Bottom-left: Table
fig.add_trace(
    go.Table(
        header=dict(
            values=["Category", "Total Investment (Taka)"],
            fill_color='burlywood',
            align='center',
            line_color='black',
            font=dict(size=16),
            line_width=2,
            height=40
        ),
        cells=dict(
            values=[table_df['Category'], table_df['Total Investment (Taka)']],
            fill_color='white',
            line_color='black',
            font=dict(size=14),
            line_width=2,
            height=35,
            align='center'
        )
    ),
    row=2, col=1
)

# Bottom-right: Pie (Participants vs Speakers)
speaker_participant_colors_map = {'Participants': 'lightgreen', 'Speakers': 'purple'}
fig.add_trace(
    go.Pie(
        labels=pie_df['Category'],
        values=pie_df['Total Investment (Taka)'],
        name="Total Investment Comparison",
        hole=0.35,
        marker=dict(colors=[speaker_participant_colors_map[c] for c in pie_df['Category']]),
    ),
    row=2, col=2
)

# Final layout
fig.update_layout(
    title="Chart 3: Investment Overview: <b>Participants vs Speakers</b> <br>(1 hour = 1000 Taka)",
    title_x=0.5,
    height=950,
    width=1400,
    showlegend=False,
    plot_bgcolor='white',
    margin=dict(l=50, r=50, b=50)
)

fig.update_xaxes(title='Speaker wise Participants', row=1, col=1)
fig.update_yaxes(title='Participant Investment (Taka)', row=1, col=1)

fig.show()


#### Long Delayed & Upcoming Courses: Months Crossed Since Last Update (> 4 Months)

In [ ]:
from datetime import datetime
import pandas as pd
import plotly.express as px

# Convert course_updated_at to datetime
df['course_updated_at'] = pd.to_datetime(df['course_updated_at'])

# Today's date
today = datetime.now()

# Approximate month difference
df['months_since_update'] = (today - df['course_updated_at']).dt.days / 30.44

# Filter: Ongoing/Upcoming + over 4 months old
filtered_df = df[
    (df['batch status'].isin(['ongoing', 'upcoming'])) &
    (df['months_since_update'] > 4)
].copy()

# Merge with speaker_df to get speaker name
filtered_df = filtered_df.merge(
    speaker_df[['id', 'name']],
    how='left',
    left_on='speaker_id',
    right_on='id',
    suffixes=('', '_speaker')
)
filtered_df.rename(columns={'name': 'speaker_name'}, inplace=True)

# Add months crossed
filtered_df['months_crossed'] = filtered_df['months_since_update'].astype(int)

# Group and retain speaker name per course-batch combo
grouped = (
    filtered_df.groupby(['coursename', 'batch status'])
    .agg(
        earliest_update=('course_updated_at', 'min'),
        months_crossed=('months_crossed', 'max'),
        speaker_name=('speaker_name', 'first')  # Get first speaker per course-batch
    )
    .reset_index()
)

# Custom text for bars
grouped['text_label'] = grouped['speaker_name'].fillna('[Unknown]') + ':\t Delay for ' + grouped['months_crossed'].astype(str) + " months"

# Sort for nicer appearance
grouped = grouped.sort_values(by='months_crossed', ascending=True)

# Plot
fig = px.bar(
    grouped,
    x='months_crossed',
    y='coursename',
    color='batch status',
    color_discrete_map=batch_color_discrete_map,
    title='Chart 4: Long Delayed & Upcoming Courses: Months Crossed Since Last Update (> 4 Months)',
    text='text_label',
    labels={'coursename': 'Course Name', 'months_crossed': 'Months Crossed Since Update'},
    hover_data={'speaker_name': True, 'earliest_update': True, 'batch status': True, 'months_crossed': True}
)

fig.update_traces(textposition='auto')
fig.update_layout(
    xaxis_title='Months Since Last Update',
    yaxis_title='Course Name',
    title_x=0.5,
    xaxis_tickangle=0,
    legend_title='Batch Status'
)

fig.show()


### Monthly Unique Course Additions Over Time

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Ensure datetime
df['start_date'] = pd.to_datetime(df['start_date'])

# === Prepare Data for Course Timeline ===
unique_courses = df.groupby('coursename')['start_date'].min().reset_index()
all_months = pd.date_range(
    start=unique_courses['start_date'].min().to_period('M').to_timestamp(),
    end=unique_courses['start_date'].max().to_period('M').to_timestamp(),
    freq='MS'
)
all_months_df = pd.DataFrame({'Month_Date': all_months})
all_months_df['Month_Label'] = all_months_df['Month_Date'].dt.strftime('%B %Y')

monthly_course_counts = unique_courses.groupby(unique_courses['start_date'].dt.to_period('M')).size().reset_index(name='Courses')
monthly_course_counts['Month_Date'] = monthly_course_counts['start_date'].dt.to_timestamp()
monthly_course_counts.drop(columns='start_date', inplace=True)

merged_course_df = pd.merge(all_months_df, monthly_course_counts, on='Month_Date', how='left').fillna(0)
merged_course_df['Courses'] = merged_course_df['Courses'].astype(int)
merged_course_df['Color'] = merged_course_df['Courses'].apply(lambda x: 'red' if x == 0 else 'green')

# === Prepare Data for Student Timeline ===
df['Month_Date'] = df['start_date'].dt.to_period('M').dt.to_timestamp()
monthly_student_counts = df.groupby('Month_Date').size().reset_index(name='Student_Count')

merged_student_df = pd.merge(all_months_df, monthly_student_counts, on='Month_Date', how='left').fillna(0)
merged_student_df['Student_Count'] = merged_student_df['Student_Count'].astype(int)
merged_student_df['Color'] = merged_student_df['Student_Count'].apply(lambda x: 'red' if x == 0 else 'blue')

# === Create Subplots ===
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Monthly New Course Additions", "Monthly Student Enrollments")
)

# --- Subplot 1: Course Additions ---
fig.add_trace(go.Scatter(
    x=merged_course_df['Month_Date'],
    y=merged_course_df['Courses'],
    mode='lines+markers',
    marker=dict(color=merged_course_df['Color'], size=8),
    line=dict(color='gray'),
    hovertemplate='<b>%{x|%B %Y}</b><br>New Courses: %{y}<extra></extra>',
    name='Courses'
), row=1, col=1)

# --- Subplot 2: Student Enrollments ---
fig.add_trace(go.Scatter(
    x=merged_student_df['Month_Date'],
    y=merged_student_df['Student_Count'],
    mode='lines+markers',
    marker=dict(color=merged_student_df['Color'], size=8),
    line=dict(color='orange'),
    hovertemplate='<b>%{x|%B %Y}</b><br>Total Enrollments: %{y}<extra></extra>',
    name='Students'
), row=2, col=1)

# === Add Date Range Annotation (Top Right Corner of First Plot) ===
start_date_str = df['start_date'].min().strftime('%d %b %Y')
end_date_str = df['start_date'].max().strftime('%d %b %Y')
date_range_text = f"<b>Date Range:</b><br>{start_date_str} to {end_date_str}"

fig.add_annotation(
    text=date_range_text,
    xref="paper", yref="paper",
    x=1.0, y=1.0,
    xanchor="right", yanchor="top",
    align="right",
    showarrow=False,
    font=dict(size=12)
)

# === Layout ===
fig.update_layout(
    height=800,
    title='Chart 5: <b>Monthly Timeline of Course Additions and Student Enrollments</b>',
    title_x=0.5,
    title_y=0.98,
    title_font=dict(size=20),
    xaxis=dict(
        tickformat='%B %Y',
        dtick='M1',
        tickangle=45
    ),
    xaxis2=dict(
        tickformat='%B %Y',
        dtick='M1',
        tickangle=45
    ),
    showlegend=False
)

fig.update_yaxes(title_text="Courses", row=1, col=1)
fig.update_yaxes(title_text="Students", row=2, col=1)

fig.show()


### Number of Unique Courses per Session Day

In [ ]:
import pandas as pd
import plotly.express as px

# Count number of **unique courses** per session day
session_day_course_counts = df.groupby('session_day')['coursename'].nunique().reset_index()
session_day_course_counts.columns = ['Day', 'Unique Courses']

# Optional: define weekday order
weekday_order = ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
session_day_course_counts['Day'] = pd.Categorical(session_day_course_counts['Day'], categories=weekday_order, ordered=True)
session_day_course_counts = session_day_course_counts.sort_values('Day')

# Plot bar chart
fig = px.bar(
    session_day_course_counts,
    x='Day',
    y='Unique Courses',
    title='Chart 6: Number of Unique Courses per Session Day',
    color='Day',
    text='Unique Courses',
    labels={'Day': 'Session Day', 'Unique Courses': 'Number of Unique Courses'}
)

# Layout tweaks
fig.update_layout(
    xaxis_title='Day of the Week',
    yaxis_title='Number of Unique Courses',
    uniformtext_minsize=8,
    title_x=0.5,
    uniformtext_mode='show',
    height=500
)

fig.show()


###  Course Duration (Participant Wise)

In [ ]:
import pandas as pd
import plotly.express as px

# Drop null durations
duration_df = df['CourseDuration'].dropna()

# Count frequencies
course_duration_counts = duration_df.value_counts().reset_index()
course_duration_counts.columns = ['CourseDuration', 'Count']

# Convert CourseDuration to total minutes for numeric sorting
def to_minutes(hhmm: str) -> int:
    try:
        h, m = hhmm.split(':')  # Split the duration into hours and minutes
        return int(h) * 60 + int(m)  # Convert to total minutes
    except ValueError:  # In case of malformed data, return a high value (to push it to the end)
        return float('inf')

# Apply conversion and create the 'TotalMinutes' column
course_duration_counts['TotalMinutes'] = course_duration_counts['CourseDuration'].apply(to_minutes)

# Sort by the TotalMinutes column to ensure chronological order
course_duration_counts = course_duration_counts.sort_values('TotalMinutes')

# Plot bar chart with the course duration ordered by time
fig = px.bar(
    course_duration_counts,
    x='CourseDuration',
    y='Count',
    color='Count',
    color_continuous_scale='RdYlGn',
    title='Chart 7: Participants Frequency of Course Durations',
    category_orders={'CourseDuration': course_duration_counts['CourseDuration'].tolist()}  # Maintain the order
)

fig.update_layout(
    xaxis_title='Course Duration',
    yaxis_title='Number of Participants',
    title_x=0.5,
    coloraxis_colorbar=dict(title='Participants')
)

fig.show()


##### Education Level vs Certification

In [ ]:
import plotly.express as px

# Get top 10 most common Last Degrees
top_10_degrees = df['Last Degree'].value_counts().head(10).index

# Filter DataFrame to only include top 10 degrees
filtered_df = df[df['Last Degree'].isin(top_10_degrees)]

# Group by 'Last Degree' and 'certificate gain', count unique participants (by Email)
degree_cert_counts = (
    filtered_df.groupby(['Last Degree', 'certificate gain'])['Email']
    .nunique()
    .reset_index(name='Unique Participant Count')
)

# Plot grouped bar chart
fig = px.bar(
    degree_cert_counts,
    x='Last Degree',
    y='Unique Participant Count',
    color='certificate gain',
    barmode='group',
    color_discrete_map=certificate_color_discrete_map,
    title='Chart 8: Top 10 Education Levels vs Certification (Unique Participants)'
)

fig.update_layout(title_x=0.5)
fig.show()


#### Participants Performance DistributionParticipant Performance Overview

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd

# Convert duration to decimal
df['CourseDurationDecimal'] = df['CourseDuration'].apply(time_to_decimal)

# ===================== Section 1: Total Courses =====================
participant_groups = df.groupby('Email').agg({
    'Name': 'first',
    'certificate gain': 'count'
}).reset_index()

participant_groups.columns = ['Email', 'Name', 'Total_Courses']
participant_groups['Display_Name'] = participant_groups['Email']

top_20_participants = participant_groups.nlargest(20, 'Total_Courses')

# ===================== Section 2: Certified Participants =====================
certified_groups = df[df['certificate gain'] == 'Certified'].groupby('Email').agg({
    'Name': 'first',
    'certificate gain': 'count'
}).reset_index()

certified_groups.columns = ['Email', 'Name', 'Certified_Count']
certified_groups['Display_Name'] = certified_groups['Email']

top_20_certified = certified_groups.nlargest(20, 'Certified_Count')

# ===================== Section 3: Course Duration =====================
filtered_df = df[(df['UserStatus'] == 'active') & (df['batch status'] == 'complete')]

email_duration = (
    filtered_df
    .groupby('Email')['CourseDurationDecimal']
    .sum()
    .reset_index()
)

name_map = filtered_df.drop_duplicates('Email')[['Email', 'Name', 'Team Name']]
email_duration = email_duration.merge(name_map, on='Email', how='left')
email_duration['Name (Team Name)'] = email_duration['Name'] + ' (' + email_duration['Team Name'] + ')'

top_duration_participants = email_duration.sort_values('CourseDurationDecimal', ascending=False).head(20)

# ===================== Subplots =====================
fig = sp.make_subplots(
    rows=3, cols=1,
    subplot_titles=[
        'Top 20 Participants by <b>Total Courses</b>',
        'Top 20 Participants by <b>Certifications</b>',
        'Top 20 Participants by <b>Total Course Duration</b>'
    ],
    vertical_spacing=0.2
)

# --- Row 1: Total Courses ---
fig.add_trace(
    go.Bar(
        x=top_20_participants['Display_Name'],
        y=top_20_participants['Total_Courses'],
        text=[f"{name}<br>{email}" for name, email in zip(top_20_participants['Name'], top_20_participants['Email'])],
        hovertemplate='<b>%{text}</b><br>Total Courses: %{y}<extra></extra>',
        marker=dict(color=top_20_participants['Total_Courses'], colorscale='RdYlGn'),
        showlegend=False
    ),
    row=1, col=1
)

# --- Row 2: Certifications ---
fig.add_trace(
    go.Bar(
        x=top_20_certified['Display_Name'],
        y=top_20_certified['Certified_Count'],
        text=[f"{name}<br>{email}" for name, email in zip(top_20_certified['Name'], top_20_certified['Email'])],
        hovertemplate='<b>%{text}</b><br>Certifications: %{y}<extra></extra>',
        marker=dict(color=top_20_certified['Certified_Count'], colorscale='RdYlGn'),
        showlegend=False
    ),
    row=2, col=1
)

# --- Row 3: Course Duration ---
fig.add_trace(
    go.Bar(
        x=top_duration_participants['Email'],
        y=top_duration_participants['CourseDurationDecimal'],
        customdata=top_duration_participants['Name'],
        text=[f"{name}<br>{email}" for name, email in zip(top_duration_participants['Name'], top_duration_participants['Email'])],
        hovertemplate='<b>%{customdata}</b><br>%{x}<br>Duration: %{y:.2f} hours<extra></extra>',
        marker=dict(color=top_duration_participants['CourseDurationDecimal'], colorscale='RdYlGn'),
        showlegend=False
    ),
    row=3, col=1
)

# ===================== Layout & Axis =====================
fig.update_layout(
    title_text="Chart 9: <b>Participant Performance Overview (Grouped by Email)</b>",
    title_x=0.5,
    height=1600,
    font=dict(size=10),
    showlegend=False
)

# X-axis angle and labels
for i in range(1, 4):
    fig.update_xaxes(tickangle=45, row=i, col=1)

# Y-axis titles
fig.update_yaxes(title_text="Total Courses", row=1, col=1)
fig.update_yaxes(title_text="Certifications", row=2, col=1)
fig.update_yaxes(title_text="Duration (Hours)", row=3, col=1)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

# --- 1. Convert course duration to decimal ---
df['CourseDurationDecimal'] = df['CourseDuration'].apply(time_to_decimal)

# --- 2. Filter only 'active' users and 'complete' batch status ---
filtered_df = df[(df['UserStatus'] == 'active') & (df['batch status'] == 'complete')]

# --- 3. Filter only Certified participants ---
certified_df = filtered_df[filtered_df['certificate gain'] == 'Certified']

# --- 4. Certified Gain Count (per participant) ---
certified_gain_counts = certified_df.groupby('Email').size().reset_index(name='Certified Gain Count')

# --- 5. Total course count (Certified + Not Certified) ---
total_counts = filtered_df.groupby('Email').size().reset_index(name='Total Count')

# --- 6. Sum of certified course durations ---
certified_duration_sum = certified_df.groupby('Email')['CourseDurationDecimal'].sum().reset_index()
certified_duration_sum.columns = ['Email', 'Total Certified Duration']

# --- 7. Merge all together ---
merged_df = certified_gain_counts.merge(total_counts, on='Email', how='left')
merged_df = merged_df.merge(certified_duration_sum, on='Email', how='left')

# --- 8. Fill NaN values to prevent errors ---
merged_df[['Certified Gain Count', 'Total Count', 'Total Certified Duration']] = \
    merged_df[['Certified Gain Count', 'Total Count', 'Total Certified Duration']].fillna(0)

# --- 9. Compute: Total Count × Certified Count × Certified Duration ---
merged_df['Multiplied Count'] = (
    merged_df['Total Count'] *
    merged_df['Certified Gain Count'] *
    merged_df['Total Certified Duration']
)

# --- 10. Normalize score to 1–100 ---
scaler = MinMaxScaler(feature_range=(1, 100))
merged_df['Normalized Multiplied Count'] = scaler.fit_transform(
    merged_df[['Multiplied Count']]
).round(2)

# --- 11. Rank participants ---
merged_df['Rank'] = merged_df['Normalized Multiplied Count'].rank(method='min', ascending=False)

# --- 12. Select Top 10 ---
top_10_filtered = merged_df.nlargest(20, 'Normalized Multiplied Count')

# --- 13. Merge Name by Email only (ensures uniqueness) ---
top_10_filtered = pd.merge(
    top_10_filtered,
    filtered_df[['Email', 'Name']].drop_duplicates(subset='Email'),
    on='Email',
    how='left'
)

# --- 14. Plot the bar chart ---
fig = px.bar(
    top_10_filtered,
    x='Name',
    y='Normalized Multiplied Count',
    title="Chart 10: <b>Top 20 Active Participants</b> <br> Performance Score = Total Course Enrollment x Certificate Gain Count x Total Duration (hrs)",
    color='Normalized Multiplied Count',
    color_continuous_scale='RdYlGn',
    labels={
        'Name': 'Participant Name',
        'Normalized Multiplied Count': 'Normalized Performance Score',
        'Total Count': 'Total Course Enrollments'
    },
    hover_data={
        'Email': True,
        'Rank': True,
        'Certified Gain Count': True,
        'Total Count': True,
        'Total Certified Duration': True
    }
)

fig.update_layout(
    xaxis_tickangle=-45,
    title_x=0.5,
    yaxis_title='Performance Score',
    xaxis_title='Participant Name'
)

fig.show()


In [ ]:
import plotly.graph_objects as go

# Prepare table data
table_df = top_10_filtered[[
    'Rank',
    'Name',
    'Email',
    'Total Count',
    'Certified Gain Count',
    'Total Certified Duration',
    'Normalized Multiplied Count'
]].copy()

# Rename columns for clarity
table_df.columns = [
    'Rank',
    'Name',
    'Email',
    'Total Course Enrollment',
    'Certificate Gain Count',
    'Total Duration (hrs)',
    'Performance Score (Normalized)'
]

# Round numeric values for cleaner table view
table_df['Total Duration (hrs)'] = table_df['Total Duration (hrs)'].round(2)
table_df['Performance Score (Normalized)'] = table_df['Performance Score (Normalized)'].round(2)
column_widths = [0.5, 1.2, 1.4, 1.0, 1.0, 1.0, 1.2]

# Create interactive table
fig_table = go.Figure(data=[go.Table(
    columnwidth=column_widths,
    header=dict(
        values=list(table_df.columns),
        fill_color='burlywood',
        align='center',
        line_color='black',
        font=dict(color='black', size=14),
        height=40
    ),
    cells=dict(
        values=[table_df[col] for col in table_df.columns],
        fill_color='white',
        align='center',
        line_color='black',
        font=dict(color='black', size=12),
        height=35
    )
)])

fig_table.update_layout(

    title=dict(
        text=" <b>Top 20 Active Participants</b> <br> Performance Score = Total Course Enrollment x Certificate Gain Count x Total Duration (hrs)",
        x=0.5,  # Center the title
        xanchor='center',
        pad=dict(b=50),  # Add padding below the title
        font=dict(size=20)
    ),
    margin=dict(l=10, r=10, t=200, b=10),
    height=1000,
    # width=1600
)

fig_table.show()


### Bottom 20 Participant Performance Overview

In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd

# Convert duration to decimal
df['CourseDurationDecimal'] = df['CourseDuration'].apply(time_to_decimal)

# ===================== Section 1: Total Courses =====================
participant_groups = df[(df['Type'] == 'Internal') & (df['UserStatus'] == 'active')].groupby('Email').agg({
    'Name': 'first',
    'certificate gain': 'count'
}).reset_index()

participant_groups.columns = ['Email', 'Name', 'Total_Courses']
participant_groups['Display_Name'] = participant_groups['Email']

bottom_20_participants = participant_groups.nsmallest(20, 'Total_Courses')

# ===================== Section 2: Certified Participants =====================
certified_groups = df[(df['certificate gain'] == 'Certified') & (df['Type'] == 'Internal') & (df['UserStatus'] == 'active') ].groupby('Email').agg({
    'Name': 'first',
    'certificate gain': 'count'
}).reset_index()

certified_groups.columns = ['Email', 'Name', 'Certified_Count']
certified_groups['Display_Name'] = certified_groups['Email']

bottom_20_certified = certified_groups.nsmallest(20, 'Certified_Count')

# ===================== Section 3: Course Duration =====================
filtered_df = df[(df['UserStatus'] == 'active') & (df['batch status'] == 'complete') & (df['Type'] == 'Internal') & (df['UserStatus'] == 'active') ]

email_duration = (
    filtered_df
    .groupby('Email')['CourseDurationDecimal']
    .sum()
    .reset_index()
)

name_map = filtered_df.drop_duplicates('Email')[['Email', 'Name', 'Team Name']]
email_duration = email_duration.merge(name_map, on='Email', how='left')
email_duration['Name (Team Name)'] = email_duration['Name'] + ' (' + email_duration['Team Name'] + ')'

bottom_duration_participants = email_duration.sort_values('CourseDurationDecimal', ascending=True).head(20)

# ===================== Subplots =====================
fig = sp.make_subplots(
    rows=3, cols=1,
    subplot_titles=[
        'Bottom 20 Participants by <b>Total Courses</b>',
        'Bottom 20 Participants by <b>Certifications</b>',
        'Bottom 20 Participants by <b>Total Course Duration</b>'
    ],
    vertical_spacing=0.2
)

# --- Row 1: Total Courses ---
fig.add_trace(
    go.Bar(
        x=bottom_20_participants['Display_Name'],
        y=bottom_20_participants['Total_Courses'],
        text=[f"{name}<br>{email}" for name, email in zip(bottom_20_participants['Name'], bottom_20_participants['Email'])],
        hovertemplate='<b>%{text}</b><br>Total Courses: %{y}<extra></extra>',
        marker=dict(color=bottom_20_participants['Total_Courses'], colorscale='RdYlGn'),
        showlegend=False
    ),
    row=1, col=1
)

# --- Row 2: Certifications ---
fig.add_trace(
    go.Bar(
        x=bottom_20_certified['Display_Name'],
        y=bottom_20_certified['Certified_Count'],
        text=[f"{name}<br>{email}" for name, email in zip(bottom_20_certified['Name'], bottom_20_certified['Email'])],
        hovertemplate='<b>%{text}</b><br>Certifications: %{y}<extra></extra>',
        marker=dict(color=bottom_20_certified['Certified_Count'], colorscale='RdYlGn'),
        showlegend=False
    ),
    row=2, col=1
)

# --- Row 3: Course Duration ---
fig.add_trace(
    go.Bar(
        x=bottom_duration_participants['Email'],
        y=bottom_duration_participants['CourseDurationDecimal'],
        customdata=bottom_duration_participants['Name'],
        text=[f"{name}<br>{email}" for name, email in zip(bottom_duration_participants['Name'], bottom_duration_participants['Email'])],
        hovertemplate='<b>%{customdata}</b><br>%{x}<br>Duration: %{y:.2f} hours<extra></extra>',
        marker=dict(color=bottom_duration_participants['CourseDurationDecimal'], colorscale='RdYlGn'),
        showlegend=False
    ),
    row=3, col=1
)

# ===================== Layout & Axis =====================
fig.update_layout(
    title_text="Chart 11: <b>Bottom 20 Participant Performance Overview (Grouped by Email)</b>",
    title_x=0.5,
    height=1600,
    font=dict(size=10),
    showlegend=False
)

# X-axis angle and labels
for i in range(1, 4):
    fig.update_xaxes(tickangle=45, row=i, col=1)

# Y-axis titles
fig.update_yaxes(title_text="Total Courses", row=1, col=1)
fig.update_yaxes(title_text="Certifications", row=2, col=1)
fig.update_yaxes(title_text="Duration (Hours)", row=3, col=1)

fig.show()


###  Course Popularity

In [ ]:
import plotly.express as px
import pandas as pd

# Step 1: Map certificate gain column
df['certificate_status'] = df['certificate gain'].map(lambda x: 'Certified' if x == 'Certified' else 'Not Certified')

# Step 2: Group by Course, Batch, and Email
course_batch_email = (
    df.groupby(['coursename', 'batchName', 'Email'])
    .agg(
        CourseDuration=('CourseDuration', 'first'),
        CertificateStatus=('certificate_status', 'first')
    )
    .reset_index()
)

# Step 3: Count certification status at Email level
course_batch_summary = (
    course_batch_email
    .groupby(['coursename', 'batchName'])
    .agg(
        Number_of_Participants=('Email', 'nunique'),
        CourseDuration=('CourseDuration', 'first'),
        Certified_Count=('CertificateStatus', lambda x: (x == 'Certified').sum()),
        NotCertified_Count=('CertificateStatus', lambda x: (x == 'Not Certified').sum())
    )
    .reset_index()
)

# Rename columns
course_batch_summary.columns = [
    'Course Name',
    'Batch Name',
    'Number of Participants',
    'Course Duration',
    'Certified Count',
    'Not Certified Count'
]

# Step 4: Get top 20 courses by total enrollment
top_courses = (
    course_batch_summary.groupby('Course Name')['Number of Participants']
    .sum()
    .sort_values(ascending=False)
    .head(20)
    .index
)

# Step 5: Filter top 20
course_batch_top = course_batch_summary[course_batch_summary['Course Name'].isin(top_courses)]

# Order Batch Name by the row-wise Number of Participants (desc)
batch_order = (
    course_batch_top
    .sort_values('Number of Participants', ascending=False)['Batch Name']
    .unique()
    .tolist()
)


fig = px.bar(
    course_batch_top,
    x='Number of Participants',
    y='Batch Name',
    color='Number of Participants',
    text='Course Name',
    hover_data=['Batch Name','Course Duration','Certified Count','Not Certified Count','Number of Participants','Course Name'],
    color_continuous_scale='RdYlGn',
    title='Chart 12: Top 20 Most Enrolled Courses based on Participants',
    orientation='h',
    category_orders={'Batch Name': batch_order}  # <- sorted by participants
)

fig.update_yaxes(title='Batch Name')  # largest at top
fig.update_xaxes(title='Number of Participants')
fig.update_traces(textposition='auto')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title_x=0.5, height=800)
fig.show()



In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ---------- PREP ----------
df['certificate_status'] = df['certificate gain'].map(lambda x: 'Certified' if x == 'Certified' else 'Not Certified')

course_batch_email = (
    df.groupby(['coursename', 'batchName', 'Email'])
      .agg(CourseDuration=('CourseDuration', 'first'),
           CertificateStatus=('certificate_status', 'first'))
      .reset_index()
)

course_batch_summary = (
    course_batch_email
    .groupby(['coursename', 'batchName'])
    .agg(Number_of_Participants=('Email', 'nunique'),
         CourseDuration=('CourseDuration', 'first'),
         Certified_Count=('CertificateStatus', lambda x: (x == 'Certified').sum()),
         NotCertified_Count=('CertificateStatus', lambda x: (x == 'Not Certified').sum()))
    .reset_index()
)

course_batch_summary.columns = [
    'Course Name', 'Batch Name', 'Number of Participants',
    'Course Duration', 'Certified Count', 'Not Certified Count'
]

# Total enrollment per course (for sorting & top/bottom picks)
totals = (course_batch_summary.groupby('Course Name')['Number of Participants']
          .sum()
          .sort_values(ascending=False))

top_names    = totals.head(20).index.tolist()
bottom_names = totals.tail(20).index.tolist()

top_df    = course_batch_summary[course_batch_summary['Course Name'].isin(top_names)]
bottom_df = course_batch_summary[course_batch_summary['Course Name'].isin(bottom_names)]

# Sort orders for y-axis (highest enrollment first)
top_order    = (top_df.groupby('Course Name')['Number of Participants'].sum()
                .sort_values(ascending=False).index.tolist())
bottom_order = (bottom_df.groupby('Course Name')['Number of Participants'].sum()
                .sort_values(ascending=False).index.tolist())

# ---------- SUBPLOTS ----------
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.12,
    subplot_titles=("Top 20 Most Enrolled Courses", "Bottom 20 Least Enrolled Courses")
)

# TOP bars (use a single shared coloraxis)
fig.add_trace(
    go.Bar(
        x=top_df['Number of Participants'],
        y=top_df['Course Name'],
        orientation='h',
        marker=dict(color=top_df['Number of Participants'], coloraxis='coloraxis'),
        text=top_df['Number of Participants'],
        textposition='auto',
        hovertemplate=(
            "<b>%{y}</b><br>" +
            "Participants: %{x}<br>" +
            "Batch: %{customdata[0]}<br>" +
            "Duration: %{customdata[1]}<br>" +
            "Certified: %{customdata[2]} | Not: %{customdata[3]}<extra></extra>"
        ),
        customdata=top_df[['Batch Name', 'Course Duration', 'Certified Count', 'Not Certified Count']]
    ),
    row=1, col=1
)

# BOTTOM bars (same shared coloraxis)
fig.add_trace(
    go.Bar(
        x=bottom_df['Number of Participants'],
        y=bottom_df['Course Name'],
        orientation='h',
        marker=dict(color=bottom_df['Number of Participants'], coloraxis='coloraxis'),
        text=bottom_df['Number of Participants'],
        textposition='auto',
        hovertemplate=(
            "<b>%{y}</b><br>" +
            "Participants: %{x}<br>" +
            "Batch: %{customdata[0]}<br>" +
            "Duration: %{customdata[1]}<br>" +
            "Certified: %{customdata[2]} | Not: %{customdata[3]}<extra></extra>"
        ),
        customdata=bottom_df[['Batch Name', 'Course Duration', 'Certified Count', 'Not Certified Count']]
    ),
    row=2, col=1
)

# ---------- LAYOUT / SORTING / COLORS ----------
# Sorting by total enrollment (highest at top of each subplot)
fig.update_yaxes(categoryorder='array', categoryarray=top_order,   autorange='reversed', row=1, col=1)
fig.update_yaxes(categoryorder='array', categoryarray=bottom_order, autorange='reversed', row=2, col=1)

fig.update_xaxes(title='Number of Participants', row=1, col=1)
fig.update_xaxes(title='Number of Participants', row=2, col=1)

fig.update_layout(
    height=1200,
    title="Chart 13: Enrollment Analysis: Top vs Bottom 20 Courses",
    title_x=0.5,
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    bargap=0.25,
    showlegend=False,
    # Single shared continuous color scale (fixes 'continuous color not working' in subplots)
    coloraxis=dict(
        colorscale='RdYlGn',
        colorbar_title='Participants'
    )
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Group by coursename, batch status, and batchName, counting unique Emails
course_batch_counts = (
    df.groupby(['coursename', 'batch status', 'batchName'])['Email']
    .nunique()
    .reset_index(name='Count')
)

# Get top 20 courses by total unique participants
top_courses = (
    course_batch_counts.groupby('coursename')['Count']
    .sum()
    .sort_values(ascending=False)
    .head(20)
    .index
)

# Filter to only top 20
course_batch_counts = course_batch_counts[course_batch_counts['coursename'].isin(top_courses)]

# Add total participants for sorting
course_batch_counts['TotalParticipants'] = course_batch_counts.groupby('coursename')['Count'].transform('sum')

# Sort batch status (e.g., ongoing first)
batch_status_order = ['ongoing', 'upcoming', 'complete']
course_batch_counts['batch status'] = pd.Categorical(
    course_batch_counts['batch status'],
    categories=batch_status_order,
    ordered=True
)

# Sort final data
course_batch_counts_sorted = course_batch_counts.sort_values(
    by=['TotalParticipants', 'batch status'],
    ascending=[False, False]
)

# Plot
fig = px.bar(
    course_batch_counts_sorted,
    x='Count',
    y='coursename',
    color='batch status',
    text='Count',
    hover_data=['batchName', 'batch status', 'Count'],
    color_discrete_map=batch_color_discrete_map,
    orientation='h',
    title='Chart 13: Batch Status Distribution by Courses (Top 20 Courses by Unique Participants)',
    category_orders={'coursename': course_batch_counts_sorted['coursename'].tolist()}
)

fig.update_layout(
    yaxis_title='Course Name',
    xaxis_title='Number of Unique Participants',
    barmode='stack',
    legend_title='Batch Status',
    title_x=0.5,
    height=800
)

fig.show()


### Top Courses by Participants (Speaker & Batch Wise)

In [ ]:
import pandas as pd
import plotly.express as px

# Merge df with speaker_df to get speaker name
merged_df = df.merge(speaker_df[['id', 'name']], left_on='speaker_id', right_on='id', how='left')

# Group by speaker name, course name, and batch name, then count unique participants by Email
speaker_course_batch_counts = (
    merged_df.groupby(['name', 'coursename', 'batchName'])['Email']
    .nunique()
    .reset_index(name='Total Participants')
)

# Sort and get top 20 speaker-course-batch combinations
top_speaker_courses = (
    speaker_course_batch_counts
    .sort_values(by='Total Participants', ascending=False)
    .head(20)
)

# Create course name order for sorting in chart
course_order = top_speaker_courses.sort_values(by='Total Participants', ascending=False)['coursename'].tolist()

# Plot
fig = px.bar(
    top_speaker_courses,
    x='Total Participants',
    y='coursename',
    color='Total Participants',  # Speaker name
    hover_data=['name', 'batchName'],
    orientation='h',
    text='Total Participants',
    color_continuous_scale='RdYlGn',
    title='Chart 14: Top Courses by Participants (Speaker & Batch Wise)',
    labels={'coursename': 'Course Name', 'name': 'Speaker Name', 'batchName': 'Batch Name'},
    category_orders={'coursename': course_order}  # 👈 important for descending order on y-axis
)

# Layout update
fig.update_layout(
    yaxis_title='Course Name',
    xaxis_title='Total Unique Participants',
    legend_title='Speaker Name',
    title_x=0.5,
    height=800
)

fig.show()


### Team Performance

In [ ]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

#  Step 1: Filter the DataFrame
approved_df = df[(df['Status'] == 'Approved') & (df['UserStatus'] == 'active')]

#  Step 2: Prepare subplot layout
fig = sp.make_subplots(
    rows=1, cols=2,
    horizontal_spacing=0.15,
    subplot_titles=('Total Team Participants', 'Courses Enrollment by Team')
)

#  Step 3: Team-wise participant count
team_status_counts = approved_df['Team Name'].value_counts().reset_index()
team_status_counts.columns = ['Team Name', 'Count']

#  Step 4: Team-wise unique course enrollment
team_course_counts = approved_df.groupby('Team Name')['coursename'].nunique().reset_index()
team_course_counts.columns = ['Team Name', 'Unique Courses']
team_course_counts = team_course_counts.sort_values(by='Unique Courses', ascending=False)

#  Step 5: Add first chart — Participants per Team
fig.add_trace(
    go.Bar(
        x=team_status_counts['Team Name'],
        y=team_status_counts['Count'],
        marker=dict(color=team_status_counts['Count'], colorscale='RdYlGn'),
        name='Enrollment Count'
    ),
    row=1, col=1
)

#  Step 6: Add second chart — Unique Courses per Team
fig.add_trace(
    go.Bar(
        x=team_course_counts['Team Name'],
        y=team_course_counts['Unique Courses'],
        marker=dict(color=team_course_counts['Unique Courses'], colorscale='RdYlGn'),
        name='Unique Courses'
    ),
    row=1, col=2
)

#  Step 7: Layout and labels
fig.update_layout(
    title_text="Chart 15: <b>Team Enrollment Overview</b>",
    title_x=0.5,
    showlegend=False,
    height=500
)

fig.update_xaxes(title_text="Team Name", row=1, col=1)
fig.update_xaxes(title_text="Team Name", row=1, col=2)
fig.update_yaxes(title_text="Number of Participants", row=1, col=1)
fig.update_yaxes(title_text="Number of Unique Courses", row=1, col=2)

fig.show()


In [ ]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

# Filter the DataFrame to include only rows where certificates were gained
certified_teams_df = df[(df['certificate gain'] == 'Certified') & (df['Status'] == 'Approved') & (df['UserStatus'] == 'active')]

# Count how many certificates each team received
cert_by_team_certified = certified_teams_df['Team Name'].value_counts().reset_index()
cert_by_team_certified.columns = ['Team Name', 'Certificates Gained']

# Filter the DataFrame to include only rows where certificates were not gained
certified_teams_df_not = df[(df['certificate gain'] == 'Not certified') & (df['Status'] == 'Approved') & (df['UserStatus'] == 'active')]

# Count how many certificates each team received
cert_by_team_not_certified = certified_teams_df_not['Team Name'].value_counts().reset_index()
cert_by_team_not_certified.columns = ['Team Name', 'Certificates Not Gained']

# Create a subplot with 1 row and 2 columns
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Number of <b>Certified</b> by Team', 'Number of <b>Not Certified</b> by Team'))

# Add certified bar chart with continuous color scale
fig.add_trace(
    go.Bar(
        x=cert_by_team_certified['Team Name'],
        y=cert_by_team_certified['Certificates Gained'],
        marker=dict(
            color=cert_by_team_certified['Certificates Gained'],
            colorscale='RdYlGn',
            # colorbar=dict(
            #     title='Certificates Gained',
            #     x=0.45,  # Position between subplots
            #     len=0.8
            # )
        ),
        name='Certified',
        hovertemplate='<b>%{x}</b><br>Certificates Gained: %{y}<extra></extra>'
    ),
    row=1, col=1
)

# Add not certified bar chart with continuous color scale (reversed)
fig.add_trace(
    go.Bar(
        x=cert_by_team_not_certified['Team Name'],
        y=cert_by_team_not_certified['Certificates Not Gained'],
        marker=dict(
            color=cert_by_team_not_certified['Certificates Not Gained'],
            colorscale='RdYlGn_r',  # Reversed color scale for Not Certified
            # colorbar=dict(
            #     title='Certificates Not Gained',
            #     x=1.02,  # Position to the right
            #     len=0.8
            # )
        ),
        name='Not Certified',
        hovertemplate='<b>%{x}</b><br>Certificates Not Gained: %{y}<extra></extra>'
    ),
    row=1, col=2
)

# Update layout for better readability
fig.update_layout(
    title_text="Chart 16: <b>Certification Status by Team Overview</b>",
    title_x=0.5,
    showlegend=False,
    height=500
)

# Update x-axis labels
fig.update_xaxes(title_text="Team Name", row=1, col=1)
fig.update_xaxes(title_text="Team Name", row=1, col=2)

# Update y-axis labels
fig.update_yaxes(title_text="Certificates Gained", row=1, col=1)
fig.update_yaxes(title_text="Certificates Not Gained", row=1, col=2)

fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Count Certified
certified = df[(df['certificate gain'] == 'Certified') & (df['Status'] == 'Approved') & (df['UserStatus'] == 'active')]['Team Name'].value_counts().reset_index()
certified.columns = ['Team Name', 'Certified']

# Count Not Certified
not_certified = df[(df['certificate gain'] == 'Not certified') & (df['Status'] == 'Approved') & (df['UserStatus'] == 'active')]['Team Name'].value_counts().reset_index()
not_certified.columns = ['Team Name', 'Not Certified']

# Merge both into one DataFrame
merged = pd.merge(certified, not_certified, on='Team Name', how='outer').fillna(0)

# Compute total and sort
merged['Total'] = merged['Certified'] + merged['Not Certified']
merged = merged.sort_values(by='Total', ascending=False)

# Convert to long format for stacked/grouped bar chart
long_df = pd.melt(
    merged,
    id_vars='Team Name',
    value_vars=['Certified', 'Not Certified'],
    var_name='Certification Status',
    value_name='Count'
)


# Plot
fig = px.bar(
    long_df,
    x='Team Name',
    y='Count',
    color='Certification Status',
    color_discrete_map=certificate_color_discrete_map,
    title='Chart 17: Certification Status by Team',
    barmode='group'          # or 'stack' if you prefer stacked bars
)

fig.update_layout(
    xaxis_title='Team Name',
    yaxis_title='Number of Participants',
    title_x=0.5,
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Count Certified
certified = df[(df['certificate gain'] == 'Certified') & (df['Status'] == 'Approved') & (df['UserStatus'] == 'active')]['Team Name'].value_counts().reset_index()
certified.columns = ['Team Name', 'Certified']

# Count Not Certified
not_certified = df[(df['certificate gain'] == 'Not certified') & (df['Status'] == 'Approved') & (df['UserStatus'] == 'active')]['Team Name'].value_counts().reset_index()
not_certified.columns = ['Team Name', 'Not Certified']

# Merge both into one DataFrame
merged = pd.merge(certified, not_certified, on='Team Name', how='outer').fillna(0)

# Compute total and calculate percentage
merged['Total'] = merged['Certified'] + merged['Not Certified']
merged['Pass Percentage'] = round((merged['Certified'] / merged['Total']) * 100, 2)
merged['Fail Percentage'] = round((merged['Not Certified'] / merged['Total']) * 100, 2)

# Convert to long format for stacked bar chart
long_df = pd.melt(
    merged[['Team Name', 'Pass Percentage', 'Fail Percentage']],
    id_vars='Team Name',
    value_vars=['Pass Percentage', 'Fail Percentage'],
    var_name='Certification Status',
    value_name='Percentage'
)

# Merge the original `merged` DataFrame with `long_df` to include the `Certified` and `Not Certified` columns
long_df = pd.merge(long_df, merged[['Team Name', 'Certified', 'Not Certified']], on='Team Name', how='left')


# Sort the data by Pass Percentage in descending order, using the original 'Pass Percentage' column
long_df = long_df.sort_values(by='Percentage', ascending=False)

# Plot
fig = px.bar(
    long_df,
    x='Team Name',
    y='Percentage',
    color='Certification Status',
    color_discrete_map={"Pass Percentage":"green", "Fail Percentage":"red"},
    title='Chart 18: Certification Ratio by Team',
    barmode='group',  # Stack the bars to show the total pass/fail percentage
    hover_data={
        'Team Name': True,               # Show team name
        'Certification Status': True,     # Show pass/fail status
        'Percentage': True,              # Show percentage
    }
)

fig.update_layout(
    xaxis_title='Team Name',
    yaxis_title='Percentage (%)',
    yaxis=dict(range=[0, 100]),  # Ensure the percentage is between 0 and 100
    legend_title='Certification Status',
    title_x=0.5,
)

fig.show()


In [ ]:
# prompt: Need speaker data chart teamwise. how many speakers are from teams. take data from speaker_df. there is a team_name

# Group by 'team_name' and count the number of speakers
speaker_counts_by_team = speaker_df['team_name'].value_counts().reset_index()
speaker_counts_by_team.columns = ['Team Name', 'Number of Speakers']

# Sort the data for better visualization
speaker_counts_by_team = speaker_counts_by_team.sort_values(by='Number of Speakers', ascending=False)

# Create a bar chart
fig = px.bar(
    speaker_counts_by_team,
    x='Team Name',
    y='Number of Speakers',
    color='Number of Speakers',  # Color by the number of speakers
    color_continuous_scale='RdYlGn', # Use a color scale
    title='Chart 19: Number of Speakers by Team',
    labels={'Team Name': 'Team Name', 'Number of Speakers': 'Number of Speakers'}
)

fig.update_layout(
    xaxis_title='Team Name',
    yaxis_title='Number of Speakers',
    xaxis_tickangle=45, # Rotate x-axis labels if they overlap
    legend_title='Number of Speakers',
    title_x=0.5,
)

fig.show()

### Top 20 Speakers

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
from plotly.graph_objs import Figure

# Merge df with speaker_df on speaker ID
merged_df = df.merge(speaker_df[['id', 'name']], left_on='speaker_id', right_on='id', how='left')

# Total participants
top_speakers = (
    merged_df.groupby('name').size()
    .reset_index(name='Total Participants')
    .sort_values(by='Total Participants', ascending=False)
    .head(10)
    .sort_values(by='Total Participants', ascending=True)
)

# Unique courses
unique_courses_by_speaker = (
    merged_df.groupby('name')['coursename'].nunique()
    .reset_index(name='Unique Courses')
    .sort_values(by='Unique Courses', ascending=False)
    .head(10)
    .sort_values(by='Unique Courses', ascending=True)
)

# Create subplot with extra horizontal spacing
fig = sp.make_subplots(
    rows=1,
    cols=2,
    subplot_titles=(
        'Top 10 Speakers by Total Participants Engagement',
        'Top 10 Speakers by Number of Unique Courses Delivered'
    ),
    horizontal_spacing=0.2  # <-- Increased spacing between plots
)

# First chart — Total Participants
fig.add_trace(
    dict(
        type='bar',
        x=top_speakers['Total Participants'],
        y=top_speakers['name'],
        orientation='h',
        marker=dict(
            color=top_speakers['Total Participants'],
            colorscale='RdYlGn',
        ),
        name='Total Participants',
        text=top_speakers['Total Participants'],
        hovertemplate='%{y}<br>Total Participants: %{x}<extra></extra>'
    ),
    row=1, col=1
)

# Second chart — Unique Courses
fig.add_trace(
    dict(
        type='bar',
        x=unique_courses_by_speaker['Unique Courses'],
        y=unique_courses_by_speaker['name'],
        orientation='h',
        marker=dict(
            color=unique_courses_by_speaker['Unique Courses'],
            colorscale='RdYlGn',
        ),
        name='Unique Courses',
        text=unique_courses_by_speaker['Unique Courses'],
        hovertemplate='%{y}<br>Unique Courses: %{x}<extra></extra>'
    ),
    row=1, col=2
)

# Final layout updates
fig.update_layout(
    title_text="Chart 20: <b>Speaker Performance Overview</b>",
    title_x=0.5,
    showlegend=False,
    height=600,
    width=1800
)

# Axis labels
fig.update_xaxes(title_text='Total Participants', row=1, col=1)
fig.update_yaxes(title_text='Speaker Name', row=1, col=1)

fig.update_xaxes(title_text='Unique Courses', row=1, col=2)
fig.update_yaxes(title_text='Speaker Name', row=1, col=2)

fig.show()


### Speakers by Organization

In [ ]:
org_counts = speaker_df['organization'].value_counts().reset_index()
org_counts.columns = ['Organization', 'Speakers']

fig = px.bar(
    org_counts,
    x='Speakers',
    y='Organization',
    text='Speakers',
    orientation='h',
    color='Organization',
    title='Chart 21: Speakers by Organization'
)
fig.update_layout(
    xaxis_title='Number of Speakers',
    yaxis_title='Organization',
    title_x=0.5
)
fig.show()


## Participant Segmentation Analysis

In [ ]:
# ============================================================================
# COMPLETE PARTICIPANT SEGMENTATION ANALYSIS WITH DATASET DOWNLOAD
# ============================================================================

import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 1. DOWNLOAD AND LOAD DATA
# ============================================================================

print("📥 Downloading dataset...")

# Install and download data
df = pd.read_excel(excel_path, sheet_name='batch status')
speaker_df = pd.read_excel(excel_path, sheet_name='speaker list')
cluster_df = pd.read_excel(excel_path, sheet_name='course cluster')

print("✅ Data loaded successfully!")
print(f"Main dataset shape: {df.shape}")
print(f"Speaker dataset shape: {speaker_df.shape}")
print(f"Cluster dataset shape: {cluster_df.shape}")

# ============================================================================
# 2. DATA PREPROCESSING
# ============================================================================

def time_to_decimal(time_str):
    """Convert time string to decimal hours"""
    if isinstance(time_str, str) and ':' in time_str:
        parts = time_str.strip().split(':')
        try:
            parts = list(map(int, parts))
            if len(parts) == 2:
                hours, minutes = parts
                seconds = 0
            elif len(parts) == 3:
                hours, minutes, seconds = parts
            else:
                return None
            return hours + minutes / 60 + seconds / 3600
        except ValueError:
            return None
    return None

# Data preprocessing
print("🔄 Preprocessing data...")
df['CourseDurationDecimal'] = df['CourseDuration'].apply(time_to_decimal)
df['start_date'] = pd.to_datetime(df['start_date'])
df['course_updated_at'] = pd.to_datetime(df['course_updated_at'])

print("✅ Data preprocessing complete!")

# ============================================================================
# 3. PARTICIPANT SEGMENTATION ANALYSIS
# ============================================================================

def segment_participants_advanced(df):
    """Advanced participant segmentation for personalized learning paths"""

    print("👥 Performing Advanced Participant Segmentation...")

    # Aggregate participant behavior
    participant_metrics = df.groupby('Email').agg({
        'Name': 'first',  # Get participant name
        'coursename': 'count',
        'certificate gain': lambda x: (x == 'Certified').sum(),
        'CourseDurationDecimal': 'sum',
        'Team Name': 'first',
        'Type': 'first',
        'start_date': ['min', 'max']
    }).reset_index()

    # Flatten column names
    participant_metrics.columns = ['Email', 'Name', 'Total_Courses', 'Certifications', 'Total_Hours',
                                 'Team', 'Type', 'First_Course', 'Last_Course']

    # Calculate derived metrics
    participant_metrics['Success_Rate'] = (participant_metrics['Certifications'] / participant_metrics['Total_Courses'] * 100).fillna(0)
    participant_metrics['Learning_Span_Days'] = (participant_metrics['Last_Course'] - participant_metrics['First_Course']).dt.days.fillna(0)
    participant_metrics['Learning_Velocity'] = (participant_metrics['Total_Courses'] / (participant_metrics['Learning_Span_Days'] + 1) * 30).fillna(0)  # Courses per month

    # Handle edge cases
    participant_metrics['Learning_Velocity'] = participant_metrics['Learning_Velocity'].replace([np.inf, -np.inf], 0)
    participant_metrics = participant_metrics.fillna(0)

    # Features for clustering
    clustering_features = ['Total_Courses', 'Success_Rate', 'Total_Hours', 'Learning_Velocity']
    X_cluster = participant_metrics[clustering_features].fillna(0)

    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_cluster)

    # K-means clustering
    kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
    participant_metrics['Cluster'] = kmeans.fit_predict(X_scaled)

    # Define cluster characteristics
    cluster_summary = participant_metrics.groupby('Cluster')[clustering_features].mean().round(2)

    # Assign meaningful names based on characteristics
    cluster_names = {
        0: 'Casual Learners',
        1: 'High Achievers',
        2: 'Struggling Participants',
        3: 'Power Users',
        4: 'New Joiners'
    }

    participant_metrics['Segment'] = participant_metrics['Cluster'].map(cluster_names)

    # Create segmentation visualization with name in hover
    fig_segments = px.scatter(
        participant_metrics,
        x='Total_Courses',
        y='Success_Rate',
        color='Segment',
        size='Total_Hours',
        hover_data=['Name', 'Email', 'Team', 'Type', 'Learning_Velocity', 'Certifications'],
        title='🎯 Participant Segmentation - Learning Behavior Analysis',
        labels={
            'Total_Courses': 'Total Courses Taken',
            'Success_Rate': 'Success Rate (%)',
            'Total_Hours': 'Total Learning Hours'
        }
    )

    # Update hover template for better readability
    fig_segments.update_traces(
        hovertemplate='<b>👤 %{customdata[0]}</b><br>' +  # Name
                      '📧 Email: %{customdata[1]}<br>' +
                      '🏢 Team: %{customdata[2]}<br>' +
                      '👔 Type: %{customdata[3]}<br>' +
                      '📚 Total Courses: %{x}<br>' +
                      '🎯 Success Rate: %{y:.1f}%<br>' +
                      '🏆 Certifications: %{customdata[5]}<br>' +
                      '⚡ Learning Velocity: %{customdata[4]:.2f} courses/month<br>' +
                      '⏰ Total Hours: %{marker.size:.1f}<br>' +
                      '🏷️ Segment: %{data.name}<extra></extra>'
    )

    # Improve layout
    fig_segments.update_layout(
        width=1000,
        height=600,
        title_x=0.5,
        font=dict(size=12),
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.01
        )
    )

    return fig_segments, participant_metrics, cluster_summary

# ============================================================================
# 4. EXECUTE ANALYSIS
# ============================================================================

print("\n🚀 Running Participant Segmentation Analysis...")
print("=" * 60)

# Run the segmentation
seg_fig, participant_segments, cluster_summary = segment_participants_advanced(df)

# Show the visualization
seg_fig.show()

# ============================================================================
# 5. BUSINESS INSIGHTS & SUMMARY
# ============================================================================

print("\n📊 PARTICIPANT SEGMENT ANALYSIS RESULTS")
print("=" * 60)

# Display cluster summary
print("\n📈 Segment Characteristics (Average Values):")
print(cluster_summary)

# Display segment distribution
print("\n👥 Segment Distribution:")
segment_counts = participant_segments['Segment'].value_counts()
total_participants = len(participant_segments)

for segment, count in segment_counts.items():
    percentage = (count / total_participants) * 100
    print(f"   🏷️ {segment}: {count} participants ({percentage:.1f}%)")

# Display top performers in each segment
print("\n🏆 Top Performers by Segment:")
for segment in participant_segments['Segment'].unique():
    segment_data = participant_segments[participant_segments['Segment'] == segment]
    if not segment_data.empty:
        top_performer = segment_data.loc[segment_data['Success_Rate'].idxmax()]
        print(f"   🌟 {segment}: {top_performer['Name']} ")
        print(f"      📊 Success Rate: {top_performer['Success_Rate']:.1f}%")
        print(f"      📚 Courses: {top_performer['Total_Courses']}")
        print(f"      🏢 Team: {top_performer['Team']}")
        print()

# Business recommendations for each segment
print("💡 ACTIONABLE BUSINESS RECOMMENDATIONS:")
print("=" * 60)

recommendations = {
    'Power Users': [
        "🎓 Offer advanced/leadership courses",
        "👨‍🏫 Consider them as mentors for other participants",
        "🎯 Provide specialized certification paths"
    ],
    'High Achievers': [
        "⚡ Fast-track them through curriculum",
        "🎖️ Offer challenging, high-value courses",
        "🤝 Pair them with struggling participants as mentors"
    ],
    'Casual Learners': [
        "📅 Offer flexible, self-paced options",
        "🎁 Provide incentives for course completion",
        "📱 Use micro-learning and mobile-friendly content"
    ],
    'Struggling Participants': [
        "🆘 Provide additional support and tutoring",
        "⏰ Offer extended deadlines and remedial courses",
        "👥 Create study groups and peer support networks"
    ],
    'New Joiners': [
        "🎯 Focus on onboarding and foundational courses",
        "📋 Provide clear learning paths and expectations",
        "👋 Assign dedicated support/mentors"
    ]
}

for segment, actions in recommendations.items():
    if segment in segment_counts.index:
        print(f"\n🏷️ {segment} ({segment_counts[segment]} participants):")
        for action in actions:
            print(f"   {action}")

# Summary statistics
print(f"\n📊 SUMMARY STATISTICS:")
print("=" * 30)
print(f"📈 Total Participants Analyzed: {total_participants}")
print(f"📚 Average Courses per Participant: {participant_segments['Total_Courses'].mean():.1f}")
print(f"🎯 Overall Success Rate: {participant_segments['Success_Rate'].mean():.1f}%")
print(f"⏰ Average Learning Hours: {participant_segments['Total_Hours'].mean():.1f}")
print(f"⚡ Average Learning Velocity: {participant_segments['Learning_Velocity'].mean():.2f} courses/month")

print("\n✅ Participant segmentation analysis complete!")
print("💡 Hover over the scatter plot points to see participant names and details.")
print("🎯 Use the recommendations above to create personalized learning experiences!")

# Optional: Save results to CSV
try:
    participant_segments.to_csv('participant_segmentation_results.csv', index=False)
    print("💾 Results saved to 'participant_segmentation_results.csv'")
except:
    print("ℹ️ Could not save CSV file (normal in Colab environment)")

print("\n🎊 Analysis Complete! Ready for Business Action! 🎊")

📥 Downloading dataset...
✅ Data loaded successfully!
Main dataset shape: (5208, 29)
Speaker dataset shape: (60, 13)
Cluster dataset shape: (77, 3)
🔄 Preprocessing data...
✅ Data preprocessing complete!

🚀 Running Participant Segmentation Analysis...
👥 Performing Advanced Participant Segmentation...



📊 PARTICIPANT SEGMENT ANALYSIS RESULTS

📈 Segment Characteristics (Average Values):
         Total_Courses  Success_Rate  Total_Hours  Learning_Velocity
Cluster                                                             
0                 1.01          0.00        19.23              30.16
1                 9.11         69.94        39.75               1.89
2                27.04         56.48       116.90               2.03
3                 1.07        100.00        19.59              29.55
4                 3.05         19.17        30.00               2.91

👥 Segment Distribution:
   🏷️ Casual Learners: 372 participants (41.5%)
   🏷️ High Achievers: 162 participants (18.1%)
   🏷️ New Joiners: 161 participants (18.0%)
   🏷️ Struggling Participants: 102 participants (11.4%)
   🏷️ Power Users: 99 participants (11.0%)

🏆 Top Performers by Segment:
   🌟 Casual Learners: Nusaiba Rafiq Surovi 
      📊 Success Rate: 0.0%
      📚 Courses: 1
      🏢 Team: 0

   🌟 High Achievers: Mahafuj Isl